# Ecoli 案例，新发现「P25906」 「Q46851」
> zhenkun.shi@tib.cas.cn   
> 2024-06-10

In [9]:
import sys,os
sys.path.insert(0, os.path.dirname(os.path.realpath('__file__')))
sys.path.insert(1,'../../')
sys.path.insert(1,'../../../')
sys.path.insert(1,'../methods/active-learning/')
from config import conf as cfg
from tools import uniprottool as uptool
from tools import  bioFunctionLib as bfl
import rxnrecer as production
from modules import commonfunction as cmfunc
from tqdm import tqdm
import re
from pandarallel import pandarallel # 导入pandaralle
pandarallel.initialize(progress_bar=False)
from tkinter import _flatten
import json
from tools import btools
from evaluation import evTools
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib_venn import venn2,venn3

FIRST_TIME_RUN = False # For the initial run, please set this flag to True. This will allow the program to download data from UniProt and RHEA, which may take longer depending on your internet speed.

%load_ext autoreload
%autoreload 2

INFO: Pandarallel will run on 192 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


# case_Q46851_P25906



| BiggID | UniProtID | GeneName | ReactionInDB | ReactionPred |
|--------|-----------|----------|--------------|--------------|
| b3001  | Q46851    | gpr      | -            | - hydroxyacetone + NAD(+) = H(+) + methylglyoxal + NADH  <br> - **NADP(+) + pyridoxine = H(+) + NADPH + pyridoxal**  <br> - a ribonucleoside 5'-triphosphate + RNA(n) = diphosphate + RNA(n+1) |
| b1406  | P25906    | pdxI     | NAD(+) + pyridoxine = H(+) + NADH + pyridoxal**  <br> NADP(+) + pyridoxine = H(+) + NADPH + pyridoxal            | - NAD(+) + pyridoxine = H(+) + NADH + pyridoxal  <br> - **NADP(+) + pyridoxine = H(+) + NADPH + pyridoxal** |

## 1. 序列比对

In [34]:
uniprot_ids = ['Q46851', 'P25906']
try2_uniprot_res=uptool.get_uniprot_records_by_ids(ids=uniprot_ids, batch_size=40)
bfl.table2fasta(table=try2_uniprot_res[['uniprot_id', 'seq']], file_out=f'{cfg.CASE_DIR}ecoli/case_Q46851_P25906.fasta')
aln_seq = bfl.blast2seq(seq1_id='P25906', seq1=try2_uniprot_res[try2_uniprot_res.uniprot_id=='P25906'].seq, seq2_id='Q46851', seq2=try2_uniprot_res[try2_uniprot_res.uniprot_id=='Q46851'].seq)
aln_seq

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
0,Q46851,P25906,73.077,26,7,0,39,64,39,64,6.700000e-12,40.8


In [28]:
try2_uniprot_res

,uniprot_id,reviewed,protein_name,gene_names,gene_oln,organism_name,length,ec,reaction_id,seq
0,P25906,reviewed,Pyridoxine 4-dehydrogenase (EC 1.1.1.65),pdxI ydbC b1406 JW1403,b1406 JW1403,Escherichia coli (strain K12),286,1.1.1.65,RHEA:16129 RHEA:52684,MSSNTFTLGTKSVNRLGYGAMQLAGPGVFGPPRDRHVAITVLREAL...
1,Q46851,reviewed,L-glyceraldehyde 3-phosphate reductase (GAP re...,gpr mgrA yghZ b3001 JW2970,b3001 JW2970,Escherichia coli (strain K12),346,1.1.1.-,,MVWLANPERYGQMQYRYCGKSGLRLPALSLGLWHNFGHVNALESQR...


## 2. 结构比对

In [90]:
pdb_save_dir = f'{cfg.CASE_DIR}/ecoli/case_Q46851_P25906'
# bfl.get_best_pdb('P25906', save_path=pdb_save_dir)
# bfl.get_best_pdb('Q46851', save_path=pdb_save_dir)
aln_structure = bfl.align_all_structures(pdb_dir=pdb_save_dir, output_tsv=f"{cfg.CASE_DIR}/ecoli/tmalign_results.tsv")
aln_structure

,pdb1,pdb2,tm_score_chain1,tm_score_chain2,tm_score_avg,tm_score_max,rmsd_tmalign,rmsd_pymol
0,RCSB_P25906_8tf1,RCSB_Q46851_3n6q,0.83925,0.76983,0.80454,0.83925,2.46,2.176


In [53]:
ds_train = pd.read_feather('http://172.16.8.97:9000/bucket-ecrecer/data/datasets/train.feather')

In [57]:
ds_train[ds_train.ec_number.str.contains(',')]

,id,isenzyme,isMultiFunctional,functionCounts,ec_number,ec_specific_level,date_integraged,seq,seqlength
469,P03015,True,True,2,"6.5.1.-,3.1.22.-",3,1986-07-21,MLIGYVRVSTNDQNTDLQRNALVCAGCEQIFEDKLSGTRTDRPGLK...,192
938,P00910,True,True,2,"4.1.1.48,5.3.1.24",4,1986-07-21,MQTVLAKIVADKAIWVEARKQQQPLASFQNEIQPSTRHFYDALQGA...,452
988,P00909,True,True,2,"4.1.1.48,5.3.1.24",4,1986-07-21,MMQTVLAKIVADKAIWVEARKQQQPLASFQNEVQPSTRHFYDALQG...,453
1028,P03523,True,True,4,"2.7.7.88,2.7.7.48,2.1.1.56,2.1.1.296",4,1986-07-21,MEVHDFETDEFNDFNEDDYATREFLNPDERMTYLNHADYNLNSPLI...,2109
1222,P00570,True,True,2,"2.7.4.6,2.7.4.3",4,1986-07-21,MEEKLKKTKIIFVVGGPGSGKGTQCEKIVQKYGYTHLSTGDLLRAE...,194
...,...,...,...,...,...,...,...,...,...
468971,P0DPE2,True,True,2,"2.1.1.-,3.4.24.71",4,2018-02-28,MACLGPSAQVPELPEKNCGYREVQYWDQRYQGAADSAPYEWFGDFS...,883
468996,M2XHU6,True,True,6,"2.3.1.38,3.1.2.14,2.3.1.39,1.3.1.9,4.2.1.59,2....",4,2018-02-28,MGSQHQSQHNSALIQAARDGEATLSVAFGGQGPSNLNCFNDLLELN...,1904
469028,A7TUG9,True,True,3,"2.3.1.86,1.1.1.100,2.3.1.41",4,2018-02-28,AAAWMLNGCLQVMDSRTIPANRNADNVDPALQTATHLCFPTRPVRV...,325
469038,M2YJJ3,True,True,3,"2.3.1.86,1.1.1.100,2.3.1.41",4,2018-02-28,MGQKTIKRKIQSAERPAEADVAFLASTQHSKDLCYEYDAPEEVAVE...,1692


In [78]:
def get_rxn_class_by_ec_strs(ecstr):
    
    ecs = [ec.strip() for ec in ecstr.split(',') if ec.strip()]
    first_digit = [ec.split('.')[0] for ec in ecs if ec and len(ec) > 0]
    
    rxn_class = list(set(first_digit))
    
    # print(ecs)
    # print(first_digit)
    # print(rxn_class)
    
    return rxn_class


get_rxn_class_by_ec_strs(ecstr=ds_train.iloc[468996].ec_number)    

['1', '4', '2', '3']

In [80]:
ds_train['rxn_class'] = ds_train.ec_number.apply(lambda x:get_rxn_class_by_ec_strs(x) )